In [37]:
# Imports
import pandas as pd
import numpy as np
import random

# Tests

In [38]:
headers = ["name","health","armor_class","initiative","initiative_bonus","team","group"]
turn_track = pd.DataFrame(columns=headers)
turn_track

,name,health,armor_class,initiative,initiative_bonus,team,group


In [39]:
character = {
    "name":"Mr.",
    "health":56,
    "armor_class":20,
    "initiative": 15,
    "initiative_bonus":2,
    "team":"Heroes",
    "group":"Alone"
}
turn_track.append(character,ignore_index=True)


,name,health,armor_class,initiative,initiative_bonus,team,group
0,Mr.,56,20,15,2,Heroes,Alone


In [40]:
headers = ["name","health","armor_class"]
filter_track = pd.DataFrame(columns=headers)
character = {
    "name":"Mrs.",
    "health":568,
    "armor_class":200
}
filter_track = filter_track.append(character,ignore_index=True)
# filter_track
filter_track[filter_track.columns[[True,False,True]]]

,name,armor_class
0,Mrs.,200


In [41]:
party1 = pd.read_csv(".\Party1.csv")
party1["group"] = np.nan

In [42]:
def read_import(path,import_groups=True):
    party = pd.read_csv(path,index_col='name')
    if import_groups and ('group' in list(party)):
        return party
    else:
        party["group"] = np.nan
        return party
party2=read_import(".\Party2.csv")
# party2

In [43]:
def initiative_based_group_assignment(groups:pd.DataFrame):
    df = groups.copy()
    df['total_initiative'] = df['initiative']+df['initiative_bonus']
    df.sort_values(by='total_initiative',ascending=False,inplace=True)
    df_count = {}
    team_order = df['team']
    group_placement = []
    lastTeam = None
    for team in team_order:
        if team != lastTeam :
            if team in df_count.keys() :
                df_count[team]+=1
            else :
                df_count[team]=1
            lastTeam = team
        group_placement.append(f"{team} {df_count[team]}")
    df['group']=group_placement
    return df.drop(columns="total_initiative")
party2_grouped = initiative_based_group_assignment(party2)
# party2_grouped

In [44]:
def roll(sided=20):
    return random.randint(1,sided)

def auto_initiative(groups:pd.DataFrame):
    df = groups.copy()
    df['initiative'] = df['initiative'].apply(lambda x: roll(20))
    return df
party2_random_init = auto_initiative(party2)
party2_random_init_grouped = initiative_based_group_assignment(party2_random_init)
party2_random_init_grouped

,health,armor_class,initiative,initiative_bonus,team,group,attributes
name,,,,,,,
Jaime Lannister,90,16,20,3,Lannister,Lannister 1,NaN
"Eddard ""Ned"" Stark",90,13,19,3,Stark,Stark 1,NaN
Jorah Mormont,90,16,18,3,Targaryen,Targaryen 1,NaN
Robb Stark,90,16,17,3,Stark,Stark 2,NaN
Cersei Lannister,90,16,16,3,Lannister,Lannister 2,NaN
Arya Stark,90,16,12,3,Stark,Stark 3,NaN
Daenerys Targaryen,90,16,9,3,Targaryen,Targaryen 2,NaN
Catelyn Stark,90,16,7,3,Stark,Stark 4,NaN
Robert Baratheon,90,16,5,3,Stark,Stark 4,NaN


In [45]:
def groups_list(turn_track:pd.DataFrame):
    return(turn_track['group'].unique())
# groups_list(party2_random_init_grouped)

In [80]:
def df_match_slice(df:pd.DataFrame,column,match):
    return df[df[column]==match]
group_to_split_df = df_match_slice(party2_random_init_grouped,'group','Stark 3')
# for member in group_to_split_df.index:
#     print(type(member))

In [91]:
party2_random_init_grouped.loc['Jon Snow','initiative']=10
party2_random_init_grouped

,health,armor_class,initiative,initiative_bonus,team,group,attributes
name,,,,,,,
Jaime Lannister,90,16,20,3,Lannister,Lannister 1,NaN
"Eddard ""Ned"" Stark",90,13,19,3,Stark,Stark 1,NaN
Jorah Mormont,90,16,18,3,Targaryen,Targaryen 1,NaN
Robb Stark,90,16,17,3,Stark,Stark 2,NaN
Cersei Lannister,90,16,16,3,Lannister,Lannister 2,NaN
Arya Stark,90,16,12,3,Stark,Stark 3,NaN
Daenerys Targaryen,90,16,9,3,Targaryen,Targaryen 2,NaN
Catelyn Stark,90,16,7,3,Stark,Stark 4,NaN
Robert Baratheon,90,16,5,3,Stark,Stark 4,NaN


In [48]:
def df_set_match_slice(df:pd.DataFrame,column,match,new_data):
    df_slice = df[df[column] == match].copy()
    df_slice[column] = new_data
    df_copy = df.copy()
    df_copy.update(df_slice)
    return df_copy
# group_to_split_df = df_set_match_slice(party2_random_init_grouped,'group','Targaryen 1',['Test1','Test2'])
# group_to_split_df

In [49]:
type(group_to_split_df)

pandas.core.frame.DataFrame

# Possible Classes

In [50]:
class Character:
    name : str
    health : int
    initiative : int
    team : str
    
    def __init__(self):
        self.name = ""
        self.health = 0
        self.initiative = 0
        self.team = None
    def __init__(self,name,health,initiative,team):
        self.name = name
        self.health = health
        self.initiative = initiative
        self.team = team

In [51]:
class GroupLoop:
    group : pd.DataFrame
    name : str

    def __init__(self):
        self.group = pd.DataFrame()
        self.name = "Unnamed Group"
    def __init__(self, list_of_characters,group_name):
        self.group = pd.DataFrame(list_of_characters)
        self.name = group_name
    def set_next_group(self, next_group):
        self.next_group = next_group